# Capturing data with Extracts
Extracts are the construct that allows users to capture and process data outside Ascent's pipeline infrastructure. Extract use cases include: Saving mesh data to HDF5 files, creating Cinema databases, and running custom Python analysis scripts. These examples outline how to use several of Ascent's extracts. See Ascent's [Extracts](https://ascent.readthedocs.io/en/latest/Actions/Extracts.html) docs for deeper details on Extracts.

In [ ]:
# remove this
import sys
sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
#sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

In [ ]:
# cleanup any old results
!./cleanup.sh
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helpers we use when displaying results in the notebook
from ascent_tutorial_jupyter_utils import img_display_width

In [ ]:
# create example mesh using the conduit blueprint braid helper
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      25,
                                      25,
                                      25,
                                      mesh)

## Relay Extract Example 1
### Save input mesh and all fields to Blueprint HDF5 Files

In [ ]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"

# add a relay extract that will write mesh data to 
# blueprint hdf5 files
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_all_fields"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()

In [ ]:
# list the output root file
!ls out_export_braid_all_fields*.root

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_export_braid_all_fields.cycle_000100.root

## Relay Extract Example 2
### Save input mesh and one selected field to Blueprint HDF5 Files

In [ ]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"

# add a relay extract that will write mesh data to 
# blueprint hdf5 files
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_one_field"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

#
# add fields parameter to limit the export to the just 
# the braid field
#
extracts["e1/params/fields"].append().set("braid")

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions);

a.close()

In [ ]:
# list the output root file
!ls out_export_braid_one_field*.root

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_export_braid_one_field.cycle_000100.root

## Relay Extract Example 3:
### Save contour results to Blueprint HDF5 Files

In [ ]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create a  pipeline (pl1) with a contour filter (f1)
pipelines["pl1/f1/type"] = "contour"

# extract contours where braid variable
# equals 0.2 and 0.4
contour_params = pipelines["pl1/f1/params"]
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set(iso_vals)

# add an extract to capture the pipeline result
add_act2 = actions.append()
add_act2["action"] = "add_extracts"
extracts = add_act2["extracts"]

# add an relay extract (e1) to export the pipeline result
# (pl1) to blueprint hdf5 files
extracts["e1/type"] = "relay"
extracts["e1/pipeline"]  = "pl1"
extracts["e1/params/path"] = "out_extract_braid_contour"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions);

a.close()

In [ ]:
# list the output root file
!ls out_extract_braid_contour*.root

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_extract_braid_contour.cycle_000100.root

## Cinema Extract Example
### Create a Cinema Image Database with several renders of the contour results

In [ ]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create a  pipeline (pl1) with a contour filter (f1)
pipelines["pl1/f1/type"] = "contour"

# extract contours where braid variable
# equals 0.2 and 0.4
contour_params = pipelines["pl1/f1/params"]
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set(iso_vals)

# declare a scene to render several angles of
# the pipeline result (pl1) to a Cinema Image
# database

add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
scenes["s1/plots/p1/field"] = "braid"
# select cinema path
scenes["s1/renders/r1/type"] = "cinema"
# use 5 renders in phi
scenes["s1/renders/r1/phi"] = 5
# and 5 renders in theta
scenes["s1/renders/r1/theta"] = 5
scenes["s1/renders/r1/db_name"] = "out_extract_cinema_contour"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions);

a.close()

In [ ]:
# use a web-server to view the results 
# use "Kernel->Interupt" to stop the webserver
!python -m http.server 8000